# DS2 protocol

## Matrix Generation

In [20]:
import hashlib
from sage.all import *
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler
from sage.modules.free_module_element import vector as sage_vector

# Parameters
number = 2 ** 46
next_prime_number = next_prime(number)
while next_prime_number % 8 != 5:
    next_prime_number = next_prime(next_prime_number)
print(q)
q = next_prime_number
n = 256
k, l = 2, 2
sigma = 3  # Standard deviation for Gaussian sampler
party_number = 2

# Define the ring Rq
R.<x> = PolynomialRing(ZZ)
Zq = Integers(q)
Rq = PolynomialRing(Zq, 'x').quotient(x^n + 1)

def discrete_gaussian_sampler(std_dev, ring):
    """
    Sample a polynomial from the discrete Gaussian distribution over a ring.
    """
    D = DiscreteGaussianDistributionIntegerSampler(sigma=std_dev)
    return ring([D() for _ in range(party_number)])

def commit(matrix, party_number):
    """
    Create a commitment of the matrix using the party number.
    """
    combined = str(matrix) + str(party_number)
    return hashlib.sha256(combined.encode()).hexdigest()

# H1 function representing the random oracle commitment
def H1(matrix, party_number):
    return commit(matrix, party_number)

# Generate k×l matrices using discrete Gaussian samplers
matrices = []
for _ in range(party_number):
    matrix = Matrix(Rq, k, l, lambda i, j: discrete_gaussian_sampler(sigma, Rq))
    matrices.append(matrix)
    #print(matrices)

# Commitments for each party
gn = []
for i, matrix in enumerate(matrices):
    commitment = H1(matrix, i + 1)
    gn.append(commitment)

print("Commitments stored in gn:", gn)

70368744177829
5
Commitments stored in gn: ['2def1465e96c97ccc3ef4ef19fb6b6e6c1677113c20678a81d9058b7db6aef5a', '3420fb44f0295743c1557299add721d6e9b65fa72572442a235a70d2184c3a10']


In [2]:
# 1. Upon receiving gj for all j ∈ [n − 1] send out An.
An = matrices[-1]
print("Sending out An:", An)

# 2. Upon receiving Aj for all j ∈ [n − 1]:
abort_flag = False
for j, Aj in enumerate(matrices[:-1]):
    if H1(Aj, j+1) != gn[j]:
        abort_flag = True
        break

if abort_flag:
    print("Sending out: abort")
else:
    A = sum(matrices)
    I = identity_matrix(Rq, k)
    A_bar = A.augment(I)
    print("Public random matrix A¯:\n", A_bar)

Sending out An: [             2*xbar + 70368744177683 70368744177679*xbar + 70368744177683]
[               xbar + 70368744177683                       70368744177683]
Public random matrix A¯:
 [         70368744177682 70368744177683*xbar + 3                       1                       0]
[70368744177681*xbar + 2 3*xbar + 70368744177681                       0                       1]


## Key Pair Generation

In [3]:
# H2 function representing the random oracle commitment
def H2(matrix, party_number):
    return commit(matrix, party_number)

# Parameters
eta = 5  # Bound for Sη

# Sη から l+k の大きさの要素をランダムに取り出す関数
def sample_from_S_eta(eta, size):
    return [ZZ.random_element(-eta, eta+1) for _ in range(size)]

# sn の初期化とデータの格納
sn = [sample_from_S_eta(eta, l+k) for _ in range(party_number)]
print(sn)

tn = [A_bar * vector(ZZ, s) for s in sn]
print(tn)

g_prime_n = H2(tn, party_number)
print("Sending out g'n:", g_prime_n)

# 2. Upon receiving g'j for all j ∈ [n − 1] send out tn.
print("Sending out tn:", tn)

# 3. Upon receiving tj for all j ∈ [n − 1]:
abort_flag = False
t_values = [tn]  # Assuming tn is already received

# Mock storage for g'j values (for the sake of this example)
g_prime_values = [g_prime_n]  # Assuming g_prime_n is already received

for j in range(party_number-1):  # Simulating the reception of other t values
    tj = [A_bar * vector(ZZ, s) for s in sn]
    t_values.append(tj)
    
    # Mock generation and storage of g'j values
    g_prime_j = H2(tj, j+1)
    g_prime_values.append(g_prime_j)
    
    if H2(tj, j+1) != g_prime_values[j+1]:  # Check against stored g'j values
        abort_flag = True
        break

if abort_flag:
    print("Sending out: abort")
else:
    t_combined = tuple(sum(t) for t in zip(*t_values))
    print("Combined public key t:", t_combined)

# Local output for Pn
skn = sn
pk = (A, t_combined)
#print("Local output for Pn (skn, pk):", skn, pk)
print("skn", skn)
print("pk", pk)

[[0, 5, -2, 1], [5, -4, 2, 4]]
[(70368744177679*xbar + 13, 15*xbar + 70368744177670), (4*xbar + 70368744177664, 70368744177657*xbar + 26)]
Sending out g'n: f27f064d4bf031b91ebd9e23a27121e162fcd7d1091abceeb0000ee60604ce88
Sending out tn: [(70368744177679*xbar + 13, 15*xbar + 70368744177670), (4*xbar + 70368744177664, 70368744177657*xbar + 26)]
Combined public key t: ((70368744177674*xbar + 26, 30*xbar + 70368744177656), (8*xbar + 70368744177644, 70368744177630*xbar + 52))
skn [[0, 5, -2, 1], [5, -4, 2, 4]]
pk ([         70368744177682 70368744177683*xbar + 3]
[70368744177681*xbar + 2 3*xbar + 70368744177681], ((70368744177674*xbar + 26, 30*xbar + 70368744177656), (8*xbar + 70368744177644, 70368744177630*xbar + 52)))


# Protocol DS2.Signn(sid, skn, pk, µ)

## Inputs

In [4]:
# H3 function for computing the per-message commitment key
def H3(message, public_key):
    combined = str(message) + str(public_key)
    return hashlib.sha256(combined.encode()).hexdigest()

# 1. Pn receives the inputs
sid = "unique_session_id_123"  # Example session ID
used_sids = set()  # Set to keep track of used session IDs
message = "example_message"

# 2. Pn verifies that sid has not been used before
if sid in used_sids:
    print("Session ID has been used before. Protocol not executed.")
else:
    used_sids.add(sid)
    
    # 3. Pn locally computes the per-message commitment key
    ck = H3(message, pk)
    print("Per-message commitment key ck:", ck)

Per-message commitment key ck: dfe24dac067fb9dcadf0904c70ae9efa7b9b2dbd6a897ac078bbcb5c04be9697


## Signature Generation

In [13]:
import random
from sage.modules.free_module_element import vector
# Parameters
eta = 5
k, l = 2, 2
N = 256
alpha = 2  # Example value for alpha
kappa = 60  # Example value for kappa
T = kappa * eta * sqrt(N * (l + k))
sigma = alpha * T
gamma = 1.1  # Example value for gamma
B = gamma * sigma * sqrt(N * (l + k))
s = 1
t = 3
M = e^((t/alpha) + (1/(2*alpha^2)))
rn = []

# H0 function
def H0(com, message, public_key):
    combined = str(com) + str(message) + str(public_key)
    hashed = hashlib.sha256(combined.encode()).digest()
    return vector(ZZ, hashed)[:l+k]  # Convert the hash to a vector in R

# Commit function
def Commit(ck, msg, r=None):
    if r is None:  # If r is not provided, sample it from D(Sr)
        r = ZZ.random_element(-Sr, Sr + 1)
    combined = str(ck) + str(msg) + str(r)
    return hashlib.sha256(combined.encode()).hexdigest()

# Openck function
def Openck(f1_f2, wn, r):
    concatenated_list = list(zero_m) + wn
    concatenated_vector = column_matrix([concatenated_list])
    new_matrix = block_matrix([[A1hat], [A2hat]])
    new_matrix_r = new_matrix * r
    new_matrix_r_list = list(new_matrix_r)
    new_matrix_r = Matrix(5, 1, new_matrix_r_list)
    if f1_f2 == new_matrix_r + concatenated_vector and r.norm()^2 <= B:
        return 1
    else:
        return 0

Rm = vector([random.uniform(0, 1) for _ in range(l+k)])    
s = alpha * T * (sqrt(2 * pi))
    
def Ds(x, Rm, s=1.0):
    rohs_zn=exp((-pi * (x.norm()^2)^2) /  s^2)
    rohs_rm=exp((-pi * (Rm.norm()^2)^2) /  s^2)
    return rohs_zn / rohs_rm

def Dcsn_s(v, x, Rm, s=1.0):
    rohcsn_s_zn=exp((-pi * ((x-v).norm()^2)^2) /  s^2)
    rohcsn_s_rm=exp((-pi * ((Rm-v).norm()^2)^2) /  s^2)
    return rohcsn_s_zn / rohcsn_s_rm

def rejection_sample(csn_list, zn_list):
    print("Starting rejection_sampling...")  # 追加
    
    csn_result = []
    zn_result = []
    for csn, zn in zip(csn_list, zn_list):
        csn_vec = vector(ZZ, csn)
        zn_vec = vector(ZZ, zn)
        
        # 比率を計算
        ratio = Ds(zn_vec, Rm, s) / (M * Dcsn_s(csn_vec, zn_vec, Rm, s))

        # 比率と1の小さい方を選ぶ
        acceptance_probability = min(1, ratio)

        # ランダムな確率を使用してサンプルを受け入れるか拒否するかを決定
        if random.random() >= acceptance_probability:
            return "restart"
        else:
            csn_result.append(csn_vec)
            zn_result.append(zn_vec)
    
    return (csn_result, zn_result)

# Signature Generation
def signature_generation(message, skn, pk, t_values):
    print("Starting signature generation...")  # 追加
    
# 1. Compute the first message
    print("Generating yn...")  # 追加
    
    # Generate k×l matrices using discrete Gaussian samplers
    yns = [vector(Rq, [discrete_gaussian_sampler(sigma, Rq) for _ in range(l+k)]) for _ in range(party_number)]
    
    print("Calculating wn...")  # 追加
    wn = [A_bar * yn for yn in yns]  # Modified to apply for each yn in yns
    #print("wn", wn)

    print("Commitck...")  # 追加
    
    #CSetup
    m = 3
    m2 = 6
   
    #homomorphic commitment    
    #CGen
    Im = identity_matrix(Rq, m)
    Ik = identity_matrix(Rq, k)
    Z = zero_matrix(Rq, k, m)
    A1hatdash = Matrix(Rq, m, m2-m, lambda i, j: discrete_gaussian_sampler(sigma, Rq))
    A2hatdash = Matrix(Rq, k, m2-m-k, lambda i, j: discrete_gaussian_sampler(sigma, Rq))
    A1hat = Im.augment(A1hatdash)
    A2hat = Z.augment(Ik).augment(A2hatdash)

    # Commitck
    comn = []
    for _ in range(party_number):
        # generate new r when sampling
        r = vector(Rq, [discrete_gaussian_sampler(sigma, Rq) for _ in range(m2)])
        print(wn)

        zero_m = [Rq(0) for _ in range(m)]
        # zero_mを3×1の行列に変形
        zero_m_matrix = Matrix(Rq, 3, 1, zero_m)

        wn_vector = Matrix(Rq, 2, 1, wn[0])
        # これらを連結して5×1の行列を作成
        concatenated_vector = block_matrix([[zero_m_matrix], [wn_vector]])

        new_matrix = block_matrix([[A1hat], [A2hat]])
        new_matrix_r = new_matrix * r
        new_matrix_r_list = list(new_matrix_r)
        new_matrix_r = Matrix(5, 1, new_matrix_r_list)
        #print("new_matrix_r", new_matrix_r)
        #print("concatenated_vector", concatenated_vector)
        f1_f2 = new_matrix_r + concatenated_vector
        comn.append(f1_f2)
        rn.append(r)
    print("rn", rn)    

        
# 2.Upon receiving comj for all j ∈ [n − 1] compute the signature share as follows.   
    if abort_flag:
        return "abort"
    else:
        com_sum = sum(comn)
        c = H0(com_sum, message, pk)
        print("c", c)
        print("skn", skn)
        csn = [[s * ci for s, ci in zip(skn_row, c)] for skn_row in skn]
        print("csn", csn)
        zn = csn + list(yns)
        print("zn", zn)
        rejection_sample(csn, zn)
                
                
#3.Upon receiving restart from some party go to 1. Otherwise upon receiving (zj , rj ) for all j ∈ [n − 1] compute the combined signature as follows
    # a. For each j in [n-1], reconstruct wj and validate the signature share
    for j, zj in enumerate(z_values):
        print(type(A_bar))
        print(type(zj))
        print(type(c))
        print(type(t_values))
        zj = vector(ZZ, zj)
        print(type(zj))
        print(type(t_values))
        t_values = vector(ZZ, t_values)
        wj = A_bar * zj - c * t_values[j]

        for i in range(party_number):
            # comn, rn, wn から要素を取得
            comn_i = comn[i]
            rn_i = rn[i]
            wn_i = wn[i]

            # Openck のチェック
            if Openck(comn_i, rn_i, wn_i) == 0:
                return "abort"

        # 追加の条件を確認
        if zj.norm()^2 <= B:
            return "abort"

    # b. Compute z and r
    z = sum(z_values)
    r = sum(r_values)

    # If the protocol does not abort, Pn obtains a signature (com, z, r) as local output
    return (comn, z, r)


        
# Example usage
while True:
    result = signature_generation(message, skn, pk, t_values)
    if result == "abort":
        print("Protocol aborted.")
        break
    elif result == "restart":
        print("Restarting the protocol.")
        # Continue with the next iteration of the loop to restart the protocol
        continue
    else:
        comn, z, r = result
        print("Commitment com:", comn)
        print("Signature z:", z)
        print("Random value r:", r)
        break  # Successful completion, exit the loop

Starting signature generation...
Generating yn...
Calculating wn...
Commitck...
[(15837*xbar^2 + 70368744122280*xbar + 70368744146031, 70368744084603*xbar^2 + 24800*xbar + 46935), (70368744167636*xbar^2 + 42138*xbar + 5173, 24228*xbar^2 + 70368744151358*xbar + 14599)]
[(15837*xbar^2 + 70368744122280*xbar + 70368744146031, 70368744084603*xbar^2 + 24800*xbar + 46935), (70368744167636*xbar^2 + 42138*xbar + 5173, 24228*xbar^2 + 70368744151358*xbar + 14599)]
rn [(70368744147346*xbar + 70368744167521, 26441*xbar + 70368744138530, 70368744166762*xbar + 15754, 4261*xbar + 70368744147824, 19191*xbar + 70368744167059, 11708*xbar + 70368744173653), (70368744147506*xbar + 70368744171677, 70368744164126*xbar + 70368744171057, 70368744174604*xbar + 7062, 38096*xbar + 70368744169185, 70368744161395*xbar + 70368744167138, 30423*xbar + 70368744172420)]
c (61, 91, 102, 104)
skn [[0, 5, -2, 1], [5, -4, 2, 4]]
csn [[0, 455, -204, 104], [305, -364, 204, 416]]
zn [[0, 455, -204, 104], [305, -364, 204, 416],

NameError: name 'z_values' is not defined

## Verification

In [ ]:
from sage.matrix.constructor import Matrix

# Verification Algorithm
def verification_algorithm(message, signature, pk):
    com, z, r = signature
    A_bar, t_combined = pk
    
    # zをMatrix型に変換
    z_rows = []
    max_length = max(len(item) if isinstance(item, tuple) else 1 for item in z)
    for item in z:
        if isinstance(item, tuple):
            z_rows.append(list(item) + [Integer(0)]*(max_length - len(item)))
        else:
            z_rows.append([item] + [Integer(0)]*(max_length - 1))
    z_matrix = Matrix(z_rows)
    
    # t_combinedがMatrix型でない場合、Matrix型に変換（必要に応じて）
    if not isinstance(t_combined, Matrix):
        t_combined = Matrix(t_combined)
    
    ck = H3(message, pk)
    c = H0(com, message, pk)
    w = A_bar * z_matrix - c * t_combined
    if z_matrix.norm(2) <= Bn and Openck(ck, com, r, w) == 1:
        return True
    else:
        return False

# Example usage
message = "Hello_World"  # Sample message for testing
signature = (com, z, r)  # Sample signature from the provided signature algorithm output

result = verification_algorithm(message, signature, pk)
print(result)
